In [418]:
import simpy
import random
import math
import numpy

In [419]:
RANDOM_SEED = 9780

INTERARRIVAL_MEAN = 6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

CALL_MEAN = 5 
CALL_RATE = 1.0 / CALL_MEAN


SERVICE_TIME_OPERATOR1_MEAN = 12
SERVICE_TIME_OPERATOR_VARIENCE = 36
mu= math.log(SERVICE_TIME_OPERATOR1_MEAN**2/math.sqrt(SERVICE_TIME_OPERATOR_VARIENCE+SERVICE_TIME_OPERATOR1_MEAN**2))
sigma=math.sqrt(math.log(1+SERVICE_TIME_OPERATOR_VARIENCE /SERVICE_TIME_OPERATOR1_MEAN))


SERVICE_TIME_OPERATOR2_MEAN = 4
SERVICE_TIME_OPERATOR2_DEVIATION = math.sqrt(3)

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60


CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

In [420]:
# Duration of the conversation between the customer and the operator (Operator1 service time)
service_times_operator1 = []

# Duration of the conversation between the customer and the operator2 (operator2 service time)
service_times_operator2 = []

# Time spent by a customer while it waits for the operator1 (Operator1 Queue waiting time)
queue_w_times_operator1 = []

# Time spent by a customer while it waits for the operator2 (operator2 Queue waiting time)
queue_w_times_operator2 = []

# The number of customer in the answering system
queue_w_customer = []

# The number of customer in the answering system
queue_w_operator1 = []

# The number of customer in the answering system
queue_w_operator2 = []



# Simulation end time
end_time = 0;



In [421]:
class Customer(object):
    def __init__(self, name, env, operator1, operator2):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.queue_t=0
    def call(self):
        global end_time 
        
        print('%s initiated a call at %g' % (self.name, self.env.now))
        queue_w_customer.append(self.env.now - self.arrival_t)
        
        prob=random.uniform(0,10)
        print('prob',prob)
        error=random.uniform(0,10)
        print('error',error)
        print('otomatik call len',len(queue_w_customer)) 
        
        
     
        if len(queue_w_customer)<101:
            yield env.timeout(random.expovariate(CALL_RATE))
            print('%s otomatik operatorden cikti at %g' % (self.name, self.env.now))
            queue_w_customer.pop()
            self.queue_t=self.env.now
            if error>1 :
                if prob<3:
                    queue_w_operator1.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call1())

                if prob>=3:  
                    queue_w_operator2.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call2())

                
            else:
                self.end_time = self.env.now
                print('%s A caller hangs up -error- at %g' % (self.name, self.env.now))
        else:
            print('%s A caller drops without answering.-100 parallel channels- at %g' % (self.name, self.env.now))
            queue_w_customer.pop()
            print('drop len',len(queue_w_customer)) 


            
    def call1(self):
        
        with operator1.request() as req:
            yield req

            print('bekleme',self.env.now - self.queue_t)
            if((self.env.now - self.queue_t))<10:
                print('%s is assigned to a operator1 at %g' % (self.name, self.env.now))
                queue_w_times_operator1.append(self.env.now - self.queue_t)
                yield self.env.process(self.get_help1_operator1())
                print('%s is done operator1 at %g' % (self.name, self.env.now))
                self.end_time = self.env.now
            else:
                print('%s A caller hangs up-10 min- at %g' % (self.name, (self.queue_t+10)))
                queue_w_times_operator1.append(self.env.now-self.env.now+10)
                self.end_time = self.queue_t+10
            queue_w_operator1.pop()


    def call2(self):
        
        with operator2.request() as req:
            yield req

            print('bekleme',self.env.now - self.queue_t)
            if((self.env.now - self.queue_t))<10:
                print('%s is assigned to a operator2 at %g' % (self.name, self.env.now))
                queue_w_times_operator2.append(self.env.now - self.queue_t)
                yield self.env.process(self.get_help2_operator2())
                print('%s is done operator2 at %g' % (self.name, self.env.now))
                self.end_time = self.env.now
            else:
                print('%s A caller hangs up-10 min- at %g' % (self.name, self.env.now))
                queue_w_times_operator2.append(10)
                self.end_time = self.queue_t+10
            queue_w_operator2.pop()

      
            

    def get_help1_operator1(self):
        duration = numpy.random.lognormal(mu, sigma)
        while duration < 0:
            duration = numpy.random.lognormal(mu, sigma)

        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help2_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [422]:

def customer_generator(env, operator1, operator2):
    """Generate new customer that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator1, operator2)
        

In [423]:
def break_generator1(env, operator1):
    """Generate new break for operator1."""
    BREAK_TIME_COUNTER1 = 0
    
    while True:
      
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        print('waiting1', len(queue_w_operator1))

        with operator1.request() as req:
            yield req
            print('Operator1 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator1 break done at %g' % (env.now))
            BREAK_TIME_COUNTER1+=1
            print('break1',BREAK_TIME_COUNTER1)

      

In [424]:
def break_generator2(env, operator2):
    """Generate new break for operator2."""
    
    BREAK_TIME_COUNTER2 = 0
    
    while True:
        
           
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        print('waiting2', len(queue_w_operator2))

      
        with operator2.request() as req:
            yield req
            print('Operator2 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator2 break done at %g' % (env.now))
            BREAK_TIME_COUNTER2+=1
            print('break2',BREAK_TIME_COUNTER2)

In [425]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator1, operator2))
env.process(break_generator1(env, operator1))
env.process(break_generator2(env, operator2))
env.run(until=1200)

Customer 1 initiated a call at 3.31508
prob 5.993637605653995
error 3.381957220400075
otomatik call len 1
Customer 1 otomatik operatorden cikti at 5.57578
bekleme 0.0
Customer 1 is assigned to a operator2 at 5.57578
Customer 2 initiated a call at 6.18385
prob 5.460343597252942
error 9.858652929789429
otomatik call len 1
Customer 2 otomatik operatorden cikti at 6.58952
Customer 3 initiated a call at 8.65602
prob 3.452484526404228
error 1.342065909993051
otomatik call len 1
Customer 1 is done operator2 at 10.1254
bekleme 3.5359265621905562
Customer 2 is assigned to a operator2 at 10.1254
Customer 2 is done operator2 at 13.895
Customer 3 otomatik operatorden cikti at 15.646
bekleme 0.0
Customer 3 is assigned to a operator2 at 15.646
waiting1 0
Operator1 gives break at 17.5986
Customer 4 initiated a call at 17.6103
prob 6.889792730135622
error 4.527166508287751
otomatik call len 1
Operator1 break done at 20.5986
break1 1
Customer 3 is done operator2 at 21.9859
Customer 5 initiated a call a

otomatik call len 2
Customer 142 is done operator2 at 823.671
Customer 143 otomatik operatorden cikti at 823.737
bekleme 0.0
Customer 143 is assigned to a operator2 at 823.737
waiting2 1
Customer 143 is done operator2 at 829.385
Operator2 gives break at 829.385
Customer 144 otomatik operatorden cikti at 829.763
Operator2 break done at 832.385
break2 16
Customer 141 is done operator1 at 837.857
bekleme 8.094343201367792
Customer 144 is assigned to a operator1 at 837.857
Customer 145 initiated a call at 840.969
prob 0.15464374352025745
error 4.374010831314959
otomatik call len 1
Customer 145 otomatik operatorden cikti at 844.397
Customer 146 initiated a call at 845.33
prob 6.084419666642721
error 8.613969604096612
otomatik call len 1
Customer 146 otomatik operatorden cikti at 846.357
bekleme 0.0
Customer 146 is assigned to a operator2 at 846.357
Customer 146 is done operator2 at 849.127
Customer 147 initiated a call at 858.63
prob 0.5298872659285281
error 0.2690819946960443
otomatik call